In [109]:
import numpy as np
import pandas as pd
import mysql.connector
import os
from dotenv import load_dotenv
from pathlib import Path

# dotenv_path = Path('.env')
# load_dotenv(dotenv_path=dotenv_path)

#%%  Declaring all the variables


# ## setting the maximum confidence a source can have: 
# maximum_confidence_value = float(os.getenv('maximum_confidence_value'))

# ## setting  the maximum number of iterations in the algo: 
# maximum_number_of_iterations =  float(os.getenv('maximum_number_of_iterations'))

# ## setting the minimum differnce in the trustworthiness scores to be reached to end the algo
# minimum_absolute_difference =  float(os.getenv('minimum_absolute_difference'))



#%% Pre-defined variables

## Function to carry out the iterations of confidence algo 
def carry_out_iterations( data,list_of_cols,t_w,id_colname,gamma): 
    
    """
    Arguments: 
        data : the table serving as input for the confidence algo 
        list_of_cols:  list of column names that serve as unique sources for the data points 
        t_w:  array of initial confidence values (source trustworthiness) for each source. Usually set to 0.5 for each source
        id_colname: Column name with unique id value for each row
        
    Returned values: 
    t_w_df : table with the changing source trustworthiness with each iteration
    train_data_confidence : Final table with the final confidence values for each data point for each source
    
    """
        
    
    max_t_w_value =  0.975
    train_data =  data[list_of_cols].copy()
    
    train_data = train_data.loc[np.sum(~train_data.isna(),axis = 1) > 1,:]
    ## creating empty data frame with same structure as traindata to copy confidence scores 
    
    train_data_confidence =  train_data.copy()
    train_data_confidence.loc[:,:]= 0

    ## calculating (1-t(w)). Carrying out calculation required for the equation
    t_w_inv =  1- t_w
    tau_w =  -np.log(t_w_inv)

    ## creating dataframe that maintains list of confidence values through each iteration
    confidence_iterations = pd.DataFrame(columns =train_data.columns.tolist() + ['iteration'])
    t_w_df = pd.DataFrame(columns = train_data.columns)

    for iteration in range(0,100):

        for col_name in list_of_cols:
            column_matching_df=  train_data_confidence.copy()
            column_matching_df.loc[:,:]= 0
            current_source =  train_data[col_name]

            other_sources_cols = [x for x in list_of_cols if x != current_source.name]

            column_matching_df[col_name] = 1
            for col_name_others in other_sources_cols:
                column_matching_df[col_name_others] = np.where(train_data[col_name_others]==current_source,1,-1)
            column_matching_df[pd.isnull(train_data)]=0

            for col_ii in range(0,column_matching_df.shape[1]):
                column_matching_df.iloc[:,col_ii] = column_matching_df.iloc[:,col_ii] * tau_w[col_ii]

            train_data_confidence[col_name]= np.where(pd.isnull(current_source),np.nan,1/(1 + np.exp( -1 * gamma * ( column_matching_df.sum(axis=1)  ) )))


        ## maintaining record of the trusworthiness scores of websites
        t_w_prev =  t_w.copy()
        t_w_df.loc[iteration]= t_w
        t_w = train_data_confidence.mean()
        t_w [t_w >= max_t_w_value] = max_t_w_value
        t_w_inv =  1- t_w
        tau_w =  -np.log(t_w_inv)

        ## printing itertion number and the trustworthiness score
        print(iteration, np.array(t_w_prev))
        if iteration > 5:
            if np.nansum(np.abs(t_w.values - t_w_prev.values)) < 0.001:
                break
    
    train_data_confidence[id_colname] =  data[id_colname]
    
    return(t_w_df,train_data_confidence )


## Function to get the final confidence values from the source trustworthiness values 
def get_final_confidence(data,list_of_cols, column_to_check_confidence,t_w ,id_colname):
    
    """
    Arguments: 
        data : the table serving as input for the confidence algo 
        list_of_cols:  list of column names that serve as unique sources for the data points 
        column_to_check_confidence:  column for which final confidence needs to be calcuated 
        t_w: trustworthiness score for each source (final values from the iterations)
        id_colname: colum with unqiue id values for each row
        
    Returned values: 
    data  : Table which returns the final confidence scores for the required columns    
    """
        
    train_data =  data[list_of_cols].copy()
    
    column_matching_df =  train_data.copy()
    column_matching_df.loc[:,:]= 0
    
    if (np.isnan(t_w[0])):
        t_w[0] = t_w[1]
    if (np.isnan(t_w[1])):
        t_w[1]= t_w[0]
    
    
    ## calculating (1-t(w)). Carrying out calculation required for the equation
    t_w_inv =  1- t_w
    tau_w =  -np.log(t_w_inv)

    current_source =  data[column_to_check_confidence]

    other_sources_cols = [x for x in list_of_cols ]

    for col_name_others in other_sources_cols:
        column_matching_df[col_name_others] = np.where(train_data[col_name_others]==current_source,1,-1)
    column_matching_df[pd.isnull(train_data)]=0

    for col_ii in range(0,column_matching_df.shape[1]):
        column_matching_df.iloc[:,col_ii] = column_matching_df.iloc[:,col_ii] * tau_w[col_ii]

    final_conf_scores= np.where(pd.isnull(current_source),np.nan,1/(1 + np.exp( -1 * gamma * ( column_matching_df.sum(axis=1)  ) )))
       

    data['final_confidence'] = final_conf_scores

    return(data)


#%% 

In [67]:

## locations where the files with confidence scores need to be saved
save_location = 'confidence_results/'

## csv file with the SQL code repo
sql_repo_csv_location =  'sql_code_repo_v5.csv'

In [68]:

## reading the table with the SQL queries for downloading necessary tables 
codes_df = pd.read_csv(sql_repo_csv_location)
codes_df_run = codes_df.loc[codes_df.Multiple_confidence_columns == 1,: ]

In [69]:
sql_folder_location =  'SQL_dump/'

In [52]:
%%time
for table_no in codes_df_run.index:
    print('Table current :',  table_no , '\n')
    string  = codes_df.loc[table_no,'SQL Code']
    string = string.replace('\n'," ")
    string = string.replace('\t'," ")
    columns = codes_df.loc[table_no,'Columns_list']
    list_of_cols = np.array(columns.split (","))
    table_name_str  = codes_df.loc[table_no,'Parent Label']+'_'+ codes_df.loc[table_no,'Field Name']
    print(table_name_str)
    
    try:
        df = pd.read_csv(sql_folder_location + table_name_str + '.csv')
        
        df['id'] = df.index
        df['Krushak_Odisha'] = df.field.combine_first(df.self)
        no_cols =  len(list_of_cols)
        t_w = np.repeat(0.5,no_cols)
        id_colname = 'id'
        gamma = 1
        t_w_df,train_data_confidence = carry_out_iterations( df,list_of_cols,t_w,id_colname, gamma)

        column_to_check_confidence = 'Krushak_Odisha'

        data_copy = get_final_confidence(df, list_of_cols, column_to_check_confidence,t_w_df.loc[t_w_df.shape[0]-1,:] ,id_colname)

        conf_table = data_copy[['Krushak_Odisha','int_krushk_id','final_confidence']]

        conf_table.to_csv( save_location + table_name_str+str(table_no)+'.csv', encoding = "utf-8")
        
    except :
        print('Table ',table_name_str, ' does not exist')
    
    

Table current : 0 

Demographic Details_Farmer Occupation
Table  Demographic Details_Farmer Occupation  does not exist
Table current : 2 

Demographic Details_Type of Crop Cultivator
0 [0.5 0.5 0.5 0.5]
1 [0.79786271 0.74654003 0.74416046 0.76224574]
2 [0.95015719 0.86125633 0.85749835 0.89135778]
3 [0.975      0.88999965 0.88841851 0.93327403]
4 [0.975      0.88811192 0.89351388 0.94686792]
5 [0.975      0.88117158 0.89672331 0.95491179]
6 [0.975      0.87436484 0.90070085 0.96129831]
7 [0.975      0.86841475 0.90493776 0.96668341]
8 [0.975      0.86333544 0.90900284 0.97127623]
9 [0.975      0.85902779 0.91270724 0.975     ]
10 [0.975      0.85549413 0.91599603 0.975     ]
11 [0.975      0.85431105 0.91884785 0.975     ]
12 [0.975      0.85371034 0.92088276 0.975     ]
13 [0.975      0.8533316  0.92229739 0.975     ]
14 [0.975      0.85307458 0.92328266 0.975     ]
Table current : 3 

Demographic Details_Activities


<timed exec>:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


0 [0.5 0.5 0.5 0.5]
1 [       nan 0.5641782  0.62695185 0.53010552]
2 [       nan 0.57573762 0.67446032 0.51687852]
3 [       nan 0.57917438 0.69767004 0.50504116]
4 [       nan 0.58065802 0.71043353 0.49783391]
5 [       nan 0.58127742 0.7178572  0.49381342]
6 [       nan 0.58141267 0.72234605 0.49170495]
7 [       nan 0.58127207 0.72516277 0.4907102 ]
8 [       nan 0.58098065 0.72700462 0.49035533]
9 [       nan 0.58061312 0.72826734 0.49036214]
10 [       nan 0.58021335 0.7291794  0.49056786]
11 [       nan 0.57980665 0.7298743  0.49087755]
12 [       nan 0.57940721 0.73043081 0.49123613]
13 [       nan 0.5790227  0.73089589 0.49161191]
14 [       nan 0.57865694 0.73129782 0.49198695]
15 [       nan 0.57831155 0.73165391 0.49235137]
Table current : 7 

Demographic Details_Gender
Table  Demographic Details_Gender  does not exist
Table current : 11 

Demographic Details_Social Category


<timed exec>:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


0 [0.5 0.5 0.5]
1 [0.73942068        nan 0.73942068]
2 [0.84828995        nan 0.84828995]
3 [0.88107941        nan 0.88107941]
4 [0.88790548        nan 0.88790548]
5 [0.88913102        nan 0.88913102]
6 [0.88934377        nan 0.88934377]
Table current : 13 

Demographic Details_Primary Mobile number


...

0 [0.5 0.5 0.5 0.5]
1 [0.80004548        nan 0.8012766  0.80005433]
2 [0.96155891        nan 0.9620546  0.96155851]
3 [0.975   nan 0.975 0.975]
4 [0.975   nan 0.975 0.975]
5 [0.975   nan 0.975 0.975]
6 [0.975   nan 0.975 0.975]
Table current : 36 

Crop Production Details_Plot No


<timed exec>:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


0 [0.5 0.5 0.5 0.5]
1 [0.8000163         nan 0.80210996 0.80001941]
2 [0.9615168         nan 0.96251356 0.96151133]
3 [0.975   nan 0.975 0.975]
4 [0.975   nan 0.975 0.975]
5 [0.975   nan 0.975 0.975]
6 [0.975   nan 0.975 0.975]
Table current : 40 

Crop Production Details_Farmer Area under Cultivation (Acres)


<timed exec>:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


0 [0.5 0.5 0.5 0.5]
1 [0.80004548        nan 0.8012766  0.80005433]
2 [0.96155891        nan 0.9620546  0.96155851]
3 [0.975   nan 0.975 0.975]
4 [0.975   nan 0.975 0.975]
5 [0.975   nan 0.975 0.975]
6 [0.975   nan 0.975 0.975]
Table current : 47 

Crop Production Details_Access to Irrigation
Table  Crop Production Details_Access to Irrigation  does not exist
Table current : 49 

Crop Production Details_Farm Equipment,Implements
Table  Crop Production Details_Farm Equipment,Implements  does not exist
CPU times: total: 15min 16s
Wall time: 18min 14s


In [58]:
df = pd.read_csv(sql_folder_location + 'datakharifcrops' + '.csv').rename(columns= {'krushk_id':'int_krushk_id'})
df['id'] = df.index
df['Krushak_Odisha'] = df['KO']
list_of_cols = ['KO','ama_krushi','ppas','mpas']
t_w = np.repeat(0.5,no_cols)
id_colname = 'id'
gamma = 1
table_name_str = 'datakharifcrops'

C:\Users\gauta\AppData\Local\Temp\ipykernel_27456\647974077.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(sql_folder_location + 'datakharifcrops' + '.csv').rename(columns= {'krushk_id':'int_krushk_id'})


In [59]:
t_w_df,train_data_confidence = carry_out_iterations( df,list_of_cols,t_w,id_colname, gamma)

column_to_check_confidence = 'Krushak_Odisha'

data_copy = get_final_confidence(df, list_of_cols, column_to_check_confidence,t_w_df.loc[t_w_df.shape[0]-1,:] ,id_colname)

0 [0.5 0.5 0.5 0.5]
1 [0.80988975 0.86514662 0.81050114 0.80051832]
2 [0.96892174 0.975      0.96876388 0.96364552]
3 [0.975 0.975 0.975 0.975]
4 [0.975 0.975 0.975 0.975]
5 [0.975 0.975 0.975 0.975]
6 [0.975 0.975 0.975 0.975]


In [60]:
conf_table = data_copy[['Krushak_Odisha','int_krushk_id','final_confidence']]
conf_table.to_csv( save_location + table_name_str+str(table_no)+'.csv', encoding = "utf-8")

In [61]:
df = pd.read_csv(sql_folder_location + 'datarabicrops' + '.csv').rename(columns= {'krushk_id':'int_krushk_id'})
df['id'] = df.index
df['Krushak_Odisha'] = df['KO']
list_of_cols = ['KO','ama_krushi','ppas','mpas']
t_w = np.repeat(0.5,no_cols)
id_colname = 'id'
gamma = 1
table_name_str = 'datarabicrops'

C:\Users\gauta\AppData\Local\Temp\ipykernel_27456\1538114463.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(sql_folder_location + 'datarabicrops' + '.csv').rename(columns= {'krushk_id':'int_krushk_id'})


In [64]:
t_w_df,train_data_confidence = carry_out_iterations( df,list_of_cols,t_w,id_colname, gamma)

column_to_check_confidence = 'Krushak_Odisha'

data_copy = get_final_confidence(df, list_of_cols, column_to_check_confidence,t_w_df.loc[t_w_df.shape[0]-1,:] ,id_colname)

0 [0.5 0.5 0.5 0.5]
1 [0.80769488 0.84800199 0.80855235 0.80003784]
2 [0.96756684 0.975      0.967372   0.96298348]
3 [0.975 0.975 0.975 0.975]
4 [0.975 0.975 0.975 0.975]
5 [0.975 0.975 0.975 0.975]
6 [0.975 0.975 0.975 0.975]


In [65]:
conf_table = data_copy[['Krushak_Odisha','int_krushk_id','final_confidence']]
conf_table.to_csv( save_location + table_name_str+str(table_no)+'.csv', encoding = "utf-8")

In [80]:
#df = pd.read_csv(sql_folder_location + 'Demographic Details_Farmer Occupation' + '.csv').rename(columns= {'krushk_id':'int_krushk_id'})
#df['id'] = df.index

#df['Krushak_Odisha'] = df.field.combine_first(df.self)
list_of_cols = ['self','field','ppas','mpas','land_records']
no_cols =  len(list_of_cols)
t_w = np.repeat(0.5,no_cols)
id_colname = 'id'
gamma = 1
table_name_str = 'farmer_occupation'

In [81]:
t_w_df,train_data_confidence = carry_out_iterations( df,list_of_cols,t_w,id_colname, gamma)

column_to_check_confidence = 'Krushak_Odisha'

data_copy = get_final_confidence(df, list_of_cols, column_to_check_confidence,t_w_df.loc[t_w_df.shape[0]-1,:] ,id_colname)

conf_table = data_copy[['Krushak_Odisha','int_krushk_id','final_confidence']]

conf_table.to_csv( save_location + table_name_str+str(table_no)+'.csv', encoding = "utf-8")

0 [0.5 0.5 0.5 0.5 0.5]
1 [0.69976898        nan 0.82197181 0.80035364 0.70468192]
2 [0.77377595        nan 0.94407056 0.91964403 0.78664682]
3 [0.79169808        nan 0.975      0.96410683 0.81422943]
4 [0.79233002        nan 0.975      0.975      0.82476928]
5 [0.78920556        nan 0.975      0.975      0.83059654]
6 [0.78572129        nan 0.975      0.975      0.83488708]
7 [0.78258111        nan 0.975      0.975      0.83841869]
8 [0.7798748         nan 0.975      0.975      0.84142203]
9 [0.77756315        nan 0.975      0.975      0.84400032]
10 [0.77558862        nan 0.975      0.975      0.84622131]
11 [0.77389848        nan 0.975      0.975      0.84813819]
12 [0.77244822        nan 0.975      0.975      0.84979507]
13 [0.77120088        nan 0.975      0.975      0.85122913]
14 [0.77012581        nan 0.975      0.975      0.85247183]
15 [0.76919748        nan 0.975      0.975      0.85354989]
16 [0.76839454        nan 0.975      0.975      0.85448608]
17 [0.76769903        nan

In [83]:
conf_table['final_confidence'].mean()

0.7706630073987739

In [107]:
df = pd.read_csv(sql_folder_location + 'Crop Production Details_Tehsil'+ '.csv').rename(columns= {'krushk_id':'int_krushk_id'})

C:\Users\gauta\AppData\Local\Temp\ipykernel_27456\3085113500.py:1: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(sql_folder_location + 'Crop Production Details_Tehsil'+ '.csv').rename(columns= {'krushk_id':'int_krushk_id'})


In [108]:
df.loc[~pd.isnull(df['ppas']),:]

,int_krushk_id,vch_aadharno,vch_tahsil,vch_village,vch_khata_no,vch_plot_no,field,self,ppas
176,514,232169939626,11.0,78.0,42,2231,3.0,NaN,ମାହାଙ୍ଗା
177,514,232169939626,11.0,78.0,42,871,3.0,NaN,ମାହାଙ୍ଗା
383,2407,229689689504,NaN,NaN,194,627,NaN,NaN,ସୁକୁନ୍ଦା
388,2407,229689689504,NaN,NaN,335,3229/3464,NaN,NaN,ସୁକୁନ୍ଦା
391,2407,229689689504,NaN,NaN,335,621/3684,NaN,NaN,ସୁକୁନ୍ଦା
...,...,...,...,...,...,...,...,...,...
12642499,8988683,977461098175,2.0,178.0,237,786,11.0,NaN,ନିମାପଡା
12648033,9058666,968908078857,8.0,161.0,208,2329,14.0,NaN,କିଶୋର ନଗର
12648035,9058666,968908078857,8.0,161.0,208,3303,14.0,NaN,କିଶୋର ନଗର
12657922,9192455,998864353075,5.0,19.0,335,728,17.0,NaN,ବିରିଡି


In [106]:
df.loc[~pd.isnull(df.ppas),: ].groupby(['field','ppas']).size().reset_index().to_csv('land_record_ppas_district.csv',index=False)
df.loc[~pd.isnull(df.mpas),: ].groupby(['field','mpas']).size().reset_index().to_csv('land_record_mpas_district.csv',index=False)

In [86]:
# df = pd.read_csv(sql_folder_location + 'equipments' + '.csv').rename(columns= {'krushk_id':'int_krushk_id'})
# df['id'] = df.index

# df['Krushak_Odisha'] = df.field.combine_first(df.self)
list_of_cols = ['field', 'self', 'mkuy']
no_cols =  len(list_of_cols)
t_w = np.repeat(0.5,no_cols)
id_colname = 'id'
gamma = 1
table_name_str = 'equipments'

In [87]:
t_w_df,train_data_confidence = carry_out_iterations( df,list_of_cols,t_w,id_colname, gamma)

column_to_check_confidence = 'Krushak_Odisha'

data_copy = get_final_confidence(df, list_of_cols, column_to_check_confidence,t_w_df.loc[t_w_df.shape[0]-1,:] ,id_colname)

conf_table = data_copy[['Krushak_Odisha','int_krushk_id','final_confidence']]

conf_table.to_csv( save_location + table_name_str+str(table_no)+'.csv', encoding = "utf-8")

0 [0.5 0.5 0.5]
1 [0.5 nan 0.5]
2 [0.5 nan 0.5]
3 [0.5 nan 0.5]
4 [0.5 nan 0.5]
5 [0.5 nan 0.5]
6 [0.5 nan 0.5]


In [92]:
conf_table['final_confidence'].mean()

0.6632432614768473

In [147]:
df = pd.read_csv(sql_folder_location + 'Crop Production Details_District' + '.csv').rename(columns= {'krushk_id':'int_krushk_id'})

C:\Users\gauta\AppData\Local\Temp\ipykernel_27456\119833290.py:1: DtypeWarning: Columns (1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(sql_folder_location + 'Crop Production Details_District' + '.csv').rename(columns= {'krushk_id':'int_krushk_id'})


In [146]:
df.shape

(8845680, 10)

In [151]:
districts_name =  pd.read_csv('district _mapping.csv').rename(columns = {'mpas':'field_English'})

In [152]:
#districts_name.groupby(['field']).transform('field_English':'count')
districts_name = districts_name.drop_duplicates()
# districts_name['count'] = districts_name.groupby('field')['field_English'].transform('count')
# districts_name.sort_values('field')

In [154]:
df = pd.merge(df,districts_name, how = 'left' )

In [158]:
df['field'] = df['field_English']

In [160]:
df['id'] = df.index

df['Krushak_Odisha'] = df.field.combine_first(df.self)
list_of_cols = ['field', 'self', 'mpas','ppas']
no_cols =  len(list_of_cols)
t_w = np.repeat(0.5,no_cols)
id_colname = 'id'
gamma = 1
table_name_str = 'Crop Production Details_District'

In [161]:
t_w_df,train_data_confidence = carry_out_iterations( df,list_of_cols,t_w,id_colname, gamma)

column_to_check_confidence = 'Krushak_Odisha'

data_copy = get_final_confidence(df, list_of_cols, column_to_check_confidence,t_w_df.loc[t_w_df.shape[0]-1,:] ,id_colname)

conf_table = data_copy[['Krushak_Odisha','int_krushk_id','final_confidence']]

conf_table.to_csv( save_location + table_name_str+str(table_no)+'.csv', encoding = "utf-8")

0 [0.5 0.5 0.5 0.5]
1 [0.78812333        nan 0.80105037 0.78752959]
2 [0.93891966        nan 0.95979803 0.93782278]
3 [0.975   nan 0.975 0.975]
4 [0.975   nan 0.975 0.975]
5 [0.975   nan 0.975 0.975]
6 [0.975   nan 0.975 0.975]


In [162]:
conf_table['final_confidence'].mean()

0.9757596225505976

In [ ]:
transliterate_scores_sum_full_db_V3

In [163]:
df = pd.read_csv(sql_folder_location + 'transliterate_scores_sum_full_db_V3' + '.csv').rename(columns= {'krushk_id':'int_krushk_id'})

In [167]:
match_vecotor = df['match_with_old']

In [169]:
df['land_details']= df['vch_farmer_name']

In [170]:
df.loc[ df['match_with_old'] == 0, 'land_details'] = 'No match'

In [172]:

df['field'] =  df['vch_farmer_name']
df['self'] =  np.nan

In [173]:
df['id'] = df.index

df['Krushak_Odisha'] = df.field.combine_first(df.self)
list_of_cols = ['field', 'self', 'land_details']
no_cols =  len(list_of_cols)
t_w = np.repeat(0.5,no_cols)
id_colname = 'id'
gamma = 1
table_name_str = 'Crop Production Famrer_names'

In [174]:
t_w_df,train_data_confidence = carry_out_iterations( df,list_of_cols,t_w,id_colname, gamma)

column_to_check_confidence = 'Krushak_Odisha'

data_copy = get_final_confidence(df, list_of_cols, column_to_check_confidence,t_w_df.loc[t_w_df.shape[0]-1,:] ,id_colname)

conf_table = data_copy[['Krushak_Odisha','int_krushk_id','final_confidence']]

conf_table.to_csv( save_location + table_name_str+str(table_no)+'.csv', encoding = "utf-8")

0 [0.5 0.5 0.5]
1 [0.71854181        nan 0.71854181]
2 [0.81076371        nan 0.81076371]
3 [0.83905135        nan 0.83905135]
4 [0.84584214        nan 0.84584214]
5 [0.84732631        nan 0.84732631]
6 [0.84764297        nan 0.84764297]


In [176]:
conf_table['final_confidence'].mean()

0.8477101697667111

In [177]:
df  = pd.read_csv(sql_folder_location +  'Crop Production Details_Farmer Area under Cultivation (Acres)'+ '.csv').rename(columns= {'krushk_id':'int_krushk_id'})

C:\Users\gauta\AppData\Local\Temp\ipykernel_27456\3433678997.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv(sql_folder_location +  'Crop Production Details_Farmer Area under Cultivation (Acres)'+ '.csv').rename(columns= {'krushk_id':'int_krushk_id'})


In [178]:
df.head(4)

,int_krushk_id,vch_aadharno,vch_tahsil,vch_village,vch_khata_no,vch_plot_no,field,self,mpas,ppas
0,817453,000000000006,2,55,14/3,126,14/3,NaN,NaN,NaN
1,3423322,000000000010,2,57,65/77,520,65/77,NaN,NaN,NaN
2,4334351,000000000011,2,57,65/120,475/1011,65/120,NaN,NaN,NaN
3,4154236,000000000012,2,57,36,360/958,36,NaN,NaN,NaN
